<a href="https://colab.research.google.com/github/mascDriver/data-science/blob/main/GEX637_Trabalho_3_%E2%80%93_Busca_Tabu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd
import time
import os

instances = {
    "dj38.tsp": "Djibouti",
    "qa194.tsp": "Qatar",
    "uy734.tsp": "Uruguay",
    "wi29.tsp": "Western Sahara",
    "zi929.tsp": "Zimbabwe",
}

path = '/content/drive/MyDrive/GEX637/'
def create_matriz_instance(filename):
  content = []
  with open(filename) as f:
    for line in f.read().splitlines():
      if line[0].isdigit():
        content.append(line.strip())
  length_tsp = content.__len__()
  dimension = np.ndarray((length_tsp, length_tsp), dtype=int)
  for key_x, x in enumerate(content):
    id_x, x1, y1 = np.array(x.split(), dtype=float)
    for key_y, y in enumerate(content):
      id_y, x2, y2 = np.array(y.split(), dtype=float)
      dimension[key_x][key_y] = round(np.sqrt((x1-x2)**2+(y1-y2)**2))
  return dimension, length_tsp

def run_tabu(length_tsp, dimension):
  best_tsp = {'time': [], 'caminho': [], 't-medio': []}
  s = np.arange(length_tsp)
  elapsed_time_instance =  length_tsp * 0.06
  print(f"Tempo de execução da instância atual: {elapsed_time_instance*10}")
  for __ in range(10):
    tabu = []
    mandato_tabu = []
    np.random.shuffle(s)
    elapsed_time = 0
    qualidade = float('inf')
    t1 = time.perf_counter()
    while(elapsed_time < elapsed_time_instance):
      fixo = s[0]
      for _ in range(length_tsp-1):
        np.delete(s, fixo)
        s = np.roll(s, -1)
        np.concatenate((fixo, s), axis=None)
        if fixo not in tabu:
          distancia = 0
          for key, value in enumerate(s):
            if value == fixo:
              continue
            try:
              distancia += dimension[value][s[key+1]]
            except IndexError:
              distancia += dimension[value][s[0]]
        for t in tabu:
          mandato_tabu[tabu.index(t)] -= 1
          if mandato_tabu[tabu.index(t)] == 0:
            mandato_tabu.pop(tabu.index(t))
            tabu.pop(tabu.index(t))
        if distancia < qualidade:
          print(f"menor distancia: {distancia}, menor qualidade: {qualidade} encontrada na volta {__}")
          tabu.append(s[0])
          mandato_tabu.append(round(length_tsp/5))
          qualidade = distancia
          best_tsp['time'].append(distancia)
          best_tsp['caminho'] = [s]
          fixo = s[1]
          break
        elapsed_time = time.perf_counter() - t1
    best_tsp['t-medio'].append(time.perf_counter() - t1)
  return best_tsp

def main():
  resultados = []
  for filename in os.scandir(path):
    dimension, length_tsp = create_matriz_instance(filename)
    print(f"Instância a ser executada {filename.name}")
    best_tsp = run_tabu(length_tsp, dimension)
    resultados.append({
        'instancia': instances[filename.name],
        'autoria': 'Diogo.Baltazar',
        'algoritmo': 'BTA',
        'q-medio': round(np.mean(best_tsp['time'])),
        'q-desvio': round(np.std(best_tsp['time']), 2),
        't-medio': round(np.mean(best_tsp['t-medio']))
    })
    print('\n')
  pd.DataFrame(resultados).to_csv('resultados.csv', index=False)
main()

Instância a ser executada dj38.tsp
Tempo de execução da instância atual: 22.799999999999997
menor distancia: 24621, menor qualidade: inf encontrada na volta 0
menor distancia: 23958, menor qualidade: 24621 encontrada na volta 0
menor distancia: 23921, menor qualidade: 23958 encontrada na volta 0
menor distancia: 23311, menor qualidade: 23921 encontrada na volta 0
menor distancia: 28829, menor qualidade: inf encontrada na volta 1
menor distancia: 28656, menor qualidade: 28829 encontrada na volta 1
menor distancia: 28648, menor qualidade: 28656 encontrada na volta 1
menor distancia: 28269, menor qualidade: 28648 encontrada na volta 1
menor distancia: 28042, menor qualidade: 28269 encontrada na volta 1
menor distancia: 26066, menor qualidade: inf encontrada na volta 2
menor distancia: 25568, menor qualidade: 26066 encontrada na volta 2
menor distancia: 25431, menor qualidade: 25568 encontrada na volta 2
menor distancia: 26426, menor qualidade: inf encontrada na volta 3
menor distancia: 25

In [19]:
pd.read_csv('resultados.csv')

,instancia,autoria,algoritmo,q-medio,q-desvio,t-medio
0,Djibouti,Diogo.Baltazar,BTA,27217,1578.36,2
1,Qatar,Diogo.Baltazar,BTA,92749,2097.54,12
2,Uruguay,Diogo.Baltazar,BTA,1632120,11993.66,44
3,Western Sahara,Diogo.Baltazar,BTA,105592,6140.80,2
4,Zimbabwe,Diogo.Baltazar,BTA,2353792,35322.65,56
